In [1]:
import pandas as pd
import numpy as np
import util as u

data='/labs/mpsnyder/moqri/data/index/'
def get_tss():
    ezh_h9='https://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76626/suppl/GSE76626_ChIP-Seq_RPM.txt.gz' ## hg38
    tss=pd.read_table(ezh_h9,index_col=0,usecols=['Probe','Chromosome','Start','End','WT_EZH2','Strand'])
    tss.columns=['chrom', 'chromStart', 'chromEnd','s','v']
    tss=tss[(tss.chrom!='X') & (tss.chrom!='Y')]
    tss=tss[tss.index.str.contains('001')]
    tss=tss[~tss.index.str.contains('-AS')]
    tss.index=tss.index.str.strip('-001_upstream')
    print(tss.shape)
    tss['d']=tss.chromEnd-tss.chromStart
    tss['tss']=tss.chromStart+2500
    tss['g']=tss.chrom.astype(int)*10**9+tss.tss
    print(tss.d.sum()/1000000,'M')
    tss.to_csv('files/tss38.csv')
def h9_cpg():
    v='38'
    bw_url='http://smithdata.usc.edu/methbase/data/Xie-Human-2013/Human_H9/tracks_hg'+v+'/Human_H9.meth.bw'
    df=u.bw2df(bw_url,tss)
    df['g']=df.index.str.split('_').str[0].astype(int)*10**9+df.index.str.split('_').str[1].astype(int)
    tssl=(tss.chrom.astype(int)*10**9+tss.tss).values
    df['tss']=df.g.apply(lambda y:u.take_closest(tssl, y))
    df['d']=df.g-df.tss
    df['dq']=pd.qcut(df.d,11)
    print(df.shape)
    df.to_csv(data+'wg/cpgs.csv')    
def get_flank():
    df['ch']='chr'+df.index.str.split('_').str[0]
    df['b']=df.index.str.split('_').str[1].astype(int)
    df['b1']=df.b-1
    df['b2']=df.b+3
    df[['ch','b1','b2']].to_csv(data+'wg/flank_pos.bed',sep='\t',header=None,index=None)
    #!bedtools getfasta -fi ref/hg38.fa.masked -bed wg/flank_pos.bed -fo wg/flank.fasta
    flank=pd.read_table(data+'wg/flank.fasta',header=None)
    flank=flank[~flank[0].str.contains('chr')]
    df['flank']=flank[0].values
    flank.shape    

In [2]:
tss=pd.read_csv('files/tss38.csv',index_col=0)
print(tss.shape)
ezh=tss[tss.v>1]
print(ezh.shape)

(31335, 8)
(929, 8)


In [3]:
df=pd.read_csv(data+'wg/cpgs.csv',index_col=0)
df.shape

(3153046, 10)

In [6]:
df[df.g.isin(tss.g)]

,beta,g,tss,d,dq,ch,b,b1,b2,flank
1_1232265,0.000,1001232265,1001232265,0,"(-85.0, 90.0]",chr1,1232265,1232264,1232268,CCGG
1_1251334,0.875,1001251334,1001251334,0,"(-85.0, 90.0]",chr1,1251334,1251333,1251337,CCGG
1_1399527,0.000,1001399527,1001399527,0,"(-85.0, 90.0]",chr1,1399527,1399526,1399530,CCGC
1_1420245,0.800,1001420245,1001420245,0,"(-85.0, 90.0]",chr1,1420245,1420244,1420248,GCGG
1_1435671,0.000,1001435671,1001435671,0,"(-85.0, 90.0]",chr1,1435671,1435670,1435674,GCGC
...,...,...,...,...,...,...,...,...,...,...
22_46762563,0.000,22046762563,22046762563,0,"(-85.0, 90.0]",chr22,46762563,46762562,46762566,ACGG
22_46762621,0.000,22046762621,22046762621,0,"(-85.0, 90.0]",chr22,46762621,46762620,46762624,GCGC
22_49853842,0.000,22049853842,22049853842,0,"(-85.0, 90.0]",chr22,49853842,49853841,49853845,GCGC
22_49918696,0.000,22049918696,22049918696,0,"(-85.0, 90.0]",chr22,49918696,49918695,49918699,NNNN


In [ ]:
ezh['gene']=ezh.index
ezh.index=ezh.g
ezh=ezh.gene.to_dict()
ezhs=ezh.keys()

In [48]:
df['ezh']=df.g.apply(lambda x:ezh[x] if x in ezhs else np.nan)

In [53]:
df[~df.ezh.isna()]

,beta,g,tss,d,dq,ch,b,b1,b2,flank,ezh
1_4655045,0.000000,1004655045,1004655045,0,"(-85.0, 90.0]",chr1,4655045,4655044,4655048,TCGC,AJAP
1_24929877,0.000000,1024929877,1024929877,0,"(-85.0, 90.0]",chr1,24929877,24929876,24929880,CCGT,RUNX3
1_37034125,0.000000,1037034125,1037034125,0,"(-85.0, 90.0]",chr1,37034125,37034124,37034128,GCGT,GRIK3
1_121396754,0.347826,1121396754,1121396754,0,"(-85.0, 90.0]",chr1,121396754,121396753,121396757,NNNN,CH17-437K3.
1_149842736,0.000000,1149842736,1149842736,0,"(-85.0, 90.0]",chr1,149842736,149842735,149842739,GCGA,HIST2H2AA4
...,...,...,...,...,...,...,...,...,...,...,...
18_72867949,0.000000,18072867949,18072867949,0,"(-85.0, 90.0]",chr18,72867949,72867948,72867952,TCGG,NETO
19_12995608,0.000000,19012995608,19012995608,0,"(-85.0, 90.0]",chr19,12995608,12995607,12995611,NNNN,NFIX
19_16324817,0.000000,19016324817,19016324817,0,"(-85.0, 90.0]",chr19,16324817,16324816,16324820,CCGC,KLF2
21_36701564,0.000000,21036701564,21036701564,0,"(-85.0, 90.0]",chr21,36701564,36701563,36701567,GCGG,AP000697.6


In [66]:
df[df.g.isin(ezh.g)]

,beta,g,tss,d,dq,ch,b,b1,b2,flank,ezh
1_4655045,0.000000,1004655045,1004655045,0,"(-85.0, 90.0]",chr1,4655045,4655044,4655048,TCGC,AJAP
1_24929877,0.000000,1024929877,1024929877,0,"(-85.0, 90.0]",chr1,24929877,24929876,24929880,CCGT,RUNX3
1_37034125,0.000000,1037034125,1037034125,0,"(-85.0, 90.0]",chr1,37034125,37034124,37034128,GCGT,GRIK3
1_121396754,0.347826,1121396754,1121396754,0,"(-85.0, 90.0]",chr1,121396754,121396753,121396757,NNNN,CH17-437K3.
1_149842736,0.000000,1149842736,1149842736,0,"(-85.0, 90.0]",chr1,149842736,149842735,149842739,GCGA,HIST2H2AA4
...,...,...,...,...,...,...,...,...,...,...,...
18_72867949,0.000000,18072867949,18072867949,0,"(-85.0, 90.0]",chr18,72867949,72867948,72867952,TCGG,NETO
19_12995608,0.000000,19012995608,19012995608,0,"(-85.0, 90.0]",chr19,12995608,12995607,12995611,NNNN,NFIX
19_16324817,0.000000,19016324817,19016324817,0,"(-85.0, 90.0]",chr19,16324817,16324816,16324820,CCGC,KLF2
21_36701564,0.000000,21036701564,21036701564,0,"(-85.0, 90.0]",chr21,36701564,36701563,36701567,GCGG,AP000697.6
